In [1]:
!sudo rm -rf /content/project

In [2]:
%cd /content
!git clone https://github.com/STASVLAD/Tips-and-Tricks-in-DL project

/content
Cloning into 'project'...
remote: Enumerating objects: 267, done.
remote: Counting objects: 100% (267/267), done.
remote: Compressing objects: 100% (218/218), done.
remote: Total 267 (delta 89), reused 148 (delta 31), pack-reused 0
Receiving objects: 100% (267/267), 6.92 MiB | 17.76 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [3]:
import os
import cv2
import time
import json
import torch
import pandas as pd
import numpy as np
from google.colab import drive
from IPython.display import clear_output
from project.modules.detect_color import detect_color
from project.modules.affect_check import affect_check
from project.modules.check_coords import check_coords
from project.modules.get_fps import get_fps

In [4]:
!gdown --id 1waoOULCmz1PA0K4pfSkTVHsCe9GYxzte -O /content/model.pt # Alex.pt

Downloading...
From: https://drive.google.com/uc?id=1waoOULCmz1PA0K4pfSkTVHsCe9GYxzte
To: /content/model.pt
496MB [00:02, 188MB/s]


In [5]:
%cd /content/project/yolov3
%pip install -qr requirements.txt 

print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

/content/project/yolov3
Setup complete. Using torch 1.8.1+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [6]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!sudo rm -rf /content/images
!sudo rm -rf /content/project/yolov3/runs
!sudo rm -rf /content/output
!sudo rm -rf /content/video.mp4

In [8]:
%cd /content
!mkdir images
!mkdir output

/content


In [9]:
!gdown --id 1c4AOUv43CGOH3qR9axkN7J30s0TlAHbT -O /content/video.mp4 #video 0

Downloading...
From: https://drive.google.com/uc?id=1c4AOUv43CGOH3qR9axkN7J30s0TlAHbT
To: /content/video.mp4
14.6MB [00:00, 88.6MB/s]


# Замер времени

In [11]:
import time
start = time.time()
!ffmpeg -nostats -loglevel 0 -i '/content/video.mp4' -s 1280x720  '/content/images/out-%06d.jpg'

%cd /content/project/yolov3
!python detect_custom.py --weights /content/model.pt --img 640 --source /content/images/ --save-dict

frame_boxes_path = '/content/project/yolov3/runs/detect/exp/boxes.json'
with open(frame_boxes_path) as json_file:
    frame_boxes = json.load(json_file)

img_folder = '/content/images'
img_names = os.listdir(img_folder)
img_names.sort()

result = {}
keys = range(len(img_names))
result = {key:{} for key in keys}

for img_path, boxes in frame_boxes.items():
    j = 0
    for box in boxes:
        xywh = np.array(box)
        img = cv2.imread(img_path)
        h = img.shape[0]
        w = img.shape[1]
        x1 = round((xywh[0] - xywh[2] / 2) * w)
        x2 = round((xywh[0] + xywh[2] / 2) * w)
        y1 = round((xywh[1] - xywh[3] / 2) * h)
        y2 = round((xywh[1] + xywh[3] / 2) * h)
        img_crop = img[y1:y2, x1:x2]

        color = detect_color(img_crop)
        affect = affect_check([x1, y1, x2, y2], img_size=(w, h))

        img_name = os.path.split(img_path)[-1]
        frame_id = img_names.index(img_name)
        result[frame_id]["traffic_light_%d" % j] = {'coords': [x1, y1, x2, y2],
                                                    'state': color,
                                                    'affect': affect
                                                    }
        j += 1

timestr = time.strftime("%d-%m %H:%M:%S")
json_path = '/content/drive/MyDrive/results/result '+ timestr
with open(json_path  + '.json', 'w') as fp:
    json.dump(result, fp, indent=4)

path = json_path + '.json'

with open(path) as json_file:
    data = json.load(json_file)
index = {}
for i, value in enumerate(data):
    index[i] = value

data = pd.read_json(path, 'index')
coord = []
stat = []
aff = []
old = data.columns
for column in data:
    for light in data[column]:
        try:
            coord.append(light['coords'])
            stat.append(light['state'])
            aff.append(light['affect'])
        except TypeError:
            coord.append(None)
            stat.append(None)
            aff.append(None)
    data[f'{column}_coords'] = coord
    data[f'{column}_state'] = stat
    data[f'{column}_affect'] = aff
    coord = []
    stat = []
    aff = []
data = data.drop(old, axis=1)
data = data.replace({np.nan: None})

for i in range(len(index)):
    try:
        data.loc[i]
    except KeyError:
        data.loc[i] = None
data.sort_index(inplace=True)
data = data.replace({np.nan: None})

window = 6
tl_number = len(data.columns) // 3

for tl in range(tl_number):
    for i in range(len(data) - window):
        past = data.iloc[i][0 + 3*tl]
        future = data.iloc[i + 1][0 + 3*tl]
        if check_coords(past, future) == None or check_coords(past, future) == False:
            delta_result = []
            for k in range(1, window):
                delta = check_coords(data.iloc[i + k][0 + 3*tl], data.iloc[i + k + 1][0 + 3*tl])
                delta_result.append(delta)
            if np.count_nonzero(np.array(delta_result)) >= 1:
                m = delta_result.index(True)
                data.loc[i][data.columns[0 + 3*tl]] = data.loc[i + 1 + m][data.columns[0 + 3*tl]]
                data.loc[i][data.columns[1 + 3*tl]] = data.loc[i + 1 + m][data.columns[1 + 3*tl]]
                data.loc[i][data.columns[2 + 3*tl]] = data.loc[i + 1 + m][data.columns[2 + 3*tl]]
            elif not np.array(delta_result).any() != None:
                data.loc[i][data.columns[0 + 3*tl]] = None
                data.loc[i][data.columns[1 + 3*tl]] = None
                data.loc[i][data.columns[2 + 3*tl]] = None

path_out = json_path + '_post.json'

out = {}
for row in data.iterrows():
    frame_dict = {}
    r = list(row[1])
    for light in range(len(data.columns) // 3):
        if r[0+3*light] is None:
            break
        else:
            frame_dict[f'traffic_light_{light}'] = {}
            frame_dict[f'traffic_light_{light}']['coords'] = r[0+3*light]
            frame_dict[f'traffic_light_{light}']['state'] = r[1+3*light]
            frame_dict[f'traffic_light_{light}']['affect'] = bool(r[2+3*light])
    out[int(row[0])] = frame_dict

with open(path_out, "w") as outfile: 
    json.dump(out, outfile, indent = 4)
end = time.time()
print(end - start)

/content/project/yolov3
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_dict=True, save_txt=False, source='/content/images/', update=False, view_img=False, weights=['/content/model.pt'])
YOLOv3  torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 261 layers, 61922845 parameters, 0 gradients, 156.3 GFLOPS
image 1/1203 /content/images/out-000001.jpg: 384x640 2 traffic lights, Done. (0.025s)
image 2/1203 /content/images/out-000002.jpg: 384x640 1 traffic lights, Done. (0.024s)
image 3/1203 /content/images/out-000003.jpg: 384x640 1 traffic lights, Done. (0.021s)
image 4/1203 /content/images/out-000004.jpg: 384x640 1 traffic lights, Done. (0.021s)
image 5/1203 /content/images/out-000005.jpg: 384x640 1 traffic lights, Done. (0.021s)
image 6/1203 /content/images/out-000006.jpg: 384x640 1 traffic lights, Done. (0.021s)
image 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: FutureWarning: Starting with Pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only


57.4360568523407
